In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib
from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.nn.functional import binary_cross_entropy_with_logits as BCEWL
import PIL
from PIL import Image
import random
import os
import shutil

In [2]:
def force_create(folderpath):
    if os.path.exists(folderpath):
        shutil.rmtree(folderpath)
    os.mkdir(folderpath)
    
def safe_create(folderpath):
    os.mkdir(folderpath)

def get_image(train: bool, name: str, number: int):
    idx = (4 - len(str(number))) * "0" + str(number)
    path = "train"
    if not(train):
        path = "test"
    path = "CIFAR-10-images/" + path
    path += "/" + name
    path += "/" + str(idx) + ".jpg"
    img = Image.open(path)
    np_data = np.asarray(img)
    fin_data = np.swapaxes(np.swapaxes(np_data, 1, 2), 0, 1)
    return fin_data/256

def load_all_data(train: bool, name: str):
    all_data = np.zeros((5000,3,32,32))
    for i in range(5000):
        all_data[i] = get_image(train, name, i)
    return all_data
    
def np_arr_to_img(data : np.ndarray):
    if data.shape[0] == 3:
        data = np.swapaxes(np.swapaxes(data, 0, 1), 1, 2)
    res = Image.fromarray(data)
    return res

def save_image(img, filepath):
    img.save(filepath)
    
def save_np_image(data : np.ndarray, filepath):
    plt.imshow(np.swapaxes(np.swapaxes(data, 0, 1), 1, 2))
    plt.savefig(filepath)
    
def save_np_image_batch(data : np.ndarray, folderpath):
    safe_create(folderpath)
    for i,x in enumerate(data):
        filepath = folderpath + "/" + str(i) + ".jpg"
        save_np_image(x, filepath)
    
def gen_rand_img(shape):
    return np.random.uniform(0,1,size = shape)
    
res = get_image(True, "airplane", 100)
save_np_image(res, "test3.jpg")

In [3]:
IN_DIM_C, IN_DIM_H, IN_DIM_W = 3, 32, 32

class Discriminator(nn.Module):
    def __init__(self, layers, learning_rate = 1e-3):
        super().__init__()
        self.layers = layers
        self.model = nn.Sequential(*layers)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = learning_rate, betas=(0.5, 0.999))
    
    def forward(self, x):
        if len(x.size()) == 1:
            return self.model(x.unsqueeze(0))[0].squeeze(-1)
        return self.model(x).squeeze(-1)
    
class Generator(nn.Module):
    def __init__(self, layers, latent_size, learning_rate = 1e-3):
        super().__init__()
        self.layers = layers
        self.latent_size = latent_size
        self.model = nn.Sequential(*layers)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = learning_rate, betas=(0.5, 0.999))
        
    def forward(self, z):
        if len(z.size()) == 1:
            self.model(z.unsqueeze(0))[0]
        return self.model(z)
    
    def gen_latent_batch(self, B):
        return torch.randn(B, self.latent_size)
    def generate_batch(self, B):
        z = self.gen_latent_batch(B)
        x = self.forward(z)
        return x
    
class GAN(nn.Module):
    def __init__(self, layers_D, layers_G, latent_size, learning_rate_D = 1e-3, learning_rate_G = 1e-3):
        super().__init__()
        self.layers_D = layers_D
        self.layers_G = layers_G
        self.D = Discriminator(layers_D, learning_rate_D)
        self.G = Generator(layers_G, latent_size, learning_rate_G)
    
    def make_batch(self, x):
        sz = len(x.size())
        assert sz >= 1 and sz <= 4
        if sz == 1 or sz == 3:
            return x.unsqueeze(0)
        return x
    
    def generate(self, B):
        return self.G.generate_batch(B)
    def generate_single(self):
        return self.generate(1)[0]
        
    def loss_D(self, x_real):
        B = x_real.size(0)
        x_fake = self.generate(B)
        d_loss_real = BCEWL(self.D(x_real), torch.ones(B), reduction = 'mean')
        d_loss_fake = BCEWL(self.D(x_fake), torch.zeros(B), reduction = 'mean')
        d_loss = d_loss_real + d_loss_fake
        return d_loss, x_fake, d_loss_real, d_loss_fake
        
    def loss_G(self, B):
        x = self.generate(B)
        logits = self.D(x)
        g_loss = -torch.mean(F.logsigmoid(logits))
        return g_loss, x
        
    def train_D(self, x_real):
        x_real = self.make_batch(x_real)
        self.D.optimizer.zero_grad()
        loss, x_fake, loss_real, loss_fake = self.loss_D(x_real)
        loss.backward()
        self.D.optimizer.step()
        return loss, x_fake
        
    def train_G(self, B):
        self.G.optimizer.zero_grad()
        loss, x_fake = self.loss_G(B)
        loss.backward()
        self.G.optimizer.step()
        return loss, x_fake
    
    def accuracy_real(self, x_real):
        correct = torch.sum(self.D(x_real) >= 0)
        total   = len(x_real)
        return correct, total, correct/total
        
    def accuracy_fake(self, x_fake):
        correct = torch.sum(self.D(x_fake) < 0)
        total   = len(x_real)
        return correct, total, correct/total
    
    def accuracy(self, x_real, x_fake = None):
        x_real = self.make_batch(x_real)
        B = x_real.size(0)
        if x_fake is None:
            x_fake = self.generate(B)
        else:
            x_fake = self.make_batch(x_fake)
        correct_real, total_real, acc_real = self.accuracy_real(x_real)
        correct_fake, total_fake, acc_fake = self.accuracy_fake(x_fake)
        assert(total_real == total_fake and total_real == B)
        correct_total = correct_real + correct_fake
        return correct_total, B + B, correct_total/(B + B), correct_real, correct_fake

In [4]:
TRAIN_ITERS = 100000
LATENT_SIZE = 10
EXP_NAME    = "NONSAT_GAN_SHIP_GPU"
EXP_NAME    = "EXPERIMENTS/" + EXP_NAME
force_create(EXP_NAME)
EVAL_SIZE   = 10
B = 64
IN_DIM_C, IN_DIM_H, IN_DIM_W = 3, 32, 32
IN_DIMS = (IN_DIM_C, IN_DIM_H, IN_DIM_W)
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


RELU_INPLACE = True
RELU = nn.ReLU(inplace = RELU_INPLACE)
LEAKY_RELU = nn.LeakyReLU(0.1, inplace = RELU_INPLACE)

layers_D = [nn.Conv2d(3, 32, (4,4), stride = 2, padding = 1), LEAKY_RELU, 
            nn.Conv2d(32, 64, (4,4), stride = 2, padding = 1), LEAKY_RELU, 
            nn.Flatten(), 
            nn.Linear(64 * IN_DIM_H // 4 * IN_DIM_W // 4, 32 * 32), LEAKY_RELU, 
            nn.Linear(32 * 32, 1)]

layers_G = [nn.Linear(LATENT_SIZE,32 * 32), nn.BatchNorm1d(32 * 32), RELU, 
            nn.Linear(32 * 32, 64 * 8 * 8), nn.BatchNorm1d(64 * 8 * 8), RELU,
            nn.Unflatten(1,(64,8,8)), nn.PixelShuffle(2),
            nn.Conv2d(64 // 4, 32, (3,3), padding = 'same'), torch.nn.BatchNorm2d(32), RELU, nn.PixelShuffle(2),
            nn.Conv2d(32 // 4, IN_DIM_C, (3,3), padding = 'same')]

MODEL = GAN(layers_D, layers_G, LATENT_SIZE, 1e-3, 1e-3)

ALL_DATA = load_all_data(True, "ship")
print("Loaded data")

losses = []
accs = []
full_accs = []

for itr in range(TRAIN_ITERS):
    x_real = torch.from_numpy(ALL_DATA[np.random.randint(2472,2473,size = B)]).float()
    loss_D, x_fake = MODEL.train_D(x_real)
    loss_G, x_fake = MODEL.train_G(B)
    correct, total, acc, correct_real, correct_fake = MODEL.accuracy(x_real, x_fake)

    losses.append([loss_D.item(), loss_G.item()])
    accs.append(acc.item())
    full_accs.append([correct, total, acc, correct_real, correct_fake])
    
    if itr%50 == 0:
        print(itr)
        np.savetxt(EXP_NAME + "/losses.txt", np.array(losses))
        np.savetxt(EXP_NAME + "/accs.txt", np.array(accs))
        np.savetxt(EXP_NAME + "/full_accs.txt", np.array(full_accs))
        torch.save(GAN, EXP_NAME + "/GAN.txt")
        fake = MODEL.generate(EVAL_SIZE).detach().numpy()
        save_np_image_batch(fake, EXP_NAME + "/EVAL_ITR_" + str(itr))
        

Loaded data


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


0


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


50


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


9900


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

9950


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [ ]:
Initialize D (Discriminator), G (Generator)
for itr in range(iterations):
    for D_step in range(k):
        x_Data, x_G = Rand_Data_batch, Generated_batch
        loss = D_loss(x_data, x_G)
        D.backprop_on_loss(loss)
    for G_step in range(l):
        x_G = Generaeted_batch
        loss = G_loss(x_G)
        G.backprop_on_loss(loss)